# 특정 키워드로 RISS 논문 검색 후 논문 정보 크롤링하기
###### By Selenium

In [1]:
pip install selenium

Note: you may need to restart the kernel to use updated packages.


In [104]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
import time
from time import sleep

In [174]:
keyword='크롤링'  #검색용 키워드
paper=[]  #결과 저장용 리스트

In [177]:
driver = webdriver.Chrome()  #드라이버 킴

In [178]:
driver.get('http://www.riss.kr/index.do')   #riss 들어감

In [180]:
driver.find_element_by_name('query').send_keys(keyword)   #검색용 키워드 입력
driver.find_element_by_name('query').submit()   #검색용 키워드 검색

In [181]:
driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[4]/div/div/div[2]/div[1]/div[2]/a').click()   #더보기 클릭

In [161]:
total=driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[4]/div[2]/div/div[1]/dl/dd/span/span').text  #전체 논문수   

In [169]:
def store():
    list=[]  #빈 리스트
    str1=""  #저자
    str2=""  #연도
    str3=""  #제목
    str4=""  #초록
        
    #저자
    str1=driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[1]/div/p/a').text
        
    #연도
    str2=driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[2]/div[1]/ul/li[4]/div/p').text
        
    #제목
    str3=driver.find_element_by_xpath('/html/body/div/div[4]/div[4]/div/div/div/div[1]/div[1]/h3').text
             
    #초록    
    try:
        try:
            more='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div/a'   #더보기 열기 (부가정보에 1개만 존재)
            driver.find_element_by_xpath(more).send_keys(Keys.ENTER)   #더보기 클릭
            sleep(0.5)
        except:
            for k in range(1,3):  #더보기 열기
                more='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div['+str(k)+']/a'
                driver.find_element_by_xpath(more).send_keys(Keys.ENTER)   #더보기 클릭
                sleep(0.5)
            
        title='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div/p'   #국문초록만 존재
        title1='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div[1]/p'   #국문초록 1번
        title2='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div[2]/p'   #국문초록 2번

        try:
            if driver.find_element_by_xpath(title).text[0] == '국':
                abstract='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div/div[3]/p'
            elif driver.find_element_by_xpath(title1).text[0] == '국':
                abstract='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div[1]/div[3]/p'
            elif driver.find_element_by_xpath(title2).text[0] == '국':
                abstract='/html/body/div/div[4]/div[4]/div/div/div/div[2]/div[1]/div/div[2]/div[3]/p'
            str4=driver.find_element_by_xpath(abstract).text 
        except:
            str4="국문초록 없음"
            
    except:  
        str4="국문초록 없음"
                    
    list.append(str1)
    list.append(str2)
    list.append(str3)
    list.append(str4)
    paper.append(list)
    sleep(2)

In [163]:
i=2
num=0
page10=True

while(i<14):  #페이지 10단위로 넘기기
    try:
        for j in range(1,11):  #논문 객체
            try:
                title='//*[@id="divContent"]/div[2]/div/div[2]/div[2]/ul/li['+str(j)+']/div[2]/p[1]/a'  #이동
                driver.find_element_by_xpath(title).send_keys(Keys.CONTROL + '\n')   #새창열기 >> 컨트롤+엔터
                driver.switch_to.window(driver.window_handles[1])   #열린 창으로 페이지 이동
                    
                store()  #크롤링 함수                
                num=num+1
                    
                driver.close()
                driver.switch_to.window(driver.window_handles[0])   #논문 창으로 페이지 이동
                sleep(2)
                
                if(num == int(total)):  #총 논문수 동일 >> for문 탈출
                    break;
                    
            except Exception as e:   #한 페이지에 논문이 10보다 적은 경우
                print(e)
        
        if(num == int(total)):  #총 논문수 동일 >> while문 탈출
                    break;
        
        i=i+1
        page='/html/body/div[1]/div[2]/div[4]/div[2]/div/div[3]/a['+str(i)+']'
        driver.find_element_by_xpath(page).send_keys(Keys.ENTER)  #10개 다 긁었으므로 다음 페이지 이동
        
        if(page10==True and i==12): #10 페이지 넘어감 >> 리셋
            page10=False
            i=3
        elif(page10==False and i==13):  #20,  30... 페이지 넘어감 >> 리셋
            i=3
            
    except Exception as e:   
                print(e)               